# **국민연금**

In [1]:
from tqdm import trange, notebook

In [2]:
# 라이브러리 불러오기
from tqdm import trange, notebook
from selenium import webdriver # 브라우저 자동화를 위한 모듈
import requests # HTTP 요청을 보내기 위한 모듈
from bs4 import BeautifulSoup as BS # HTML 내용 파싱을 위한 모듈
import pandas as pd # 데이터 조작 및 분석을 모듈
import time # 코드 실행 속도 조절을 위한 모듈
import re # 정규 표현식 사용을 위한 모듈
import pickle # 파이썬 객체 직렬화를 위한 모듈
from selenium.webdriver.common.by import By # 다양한 방법으로 엘리먼트를 찾기 위한 모듈
import datetime # 날짜와 시간 연산을 위한 모듈
from newspaper import Article
import time
import numpy as np
import pandas as pd
from threading import Thread
from datetime import datetime

## **크롤링 속도 개선을 위한 멀티쓰레딩 개념을 이용한 크롤링 방법으로 진행**

* 멀티쓰레딩을 사용하지 않으면 한 개의 크롬창만 띄워서 크롤링을 진행하는 탓에 속도가 매우 느립니다. 하지만 멀티쓰레딩 개념을 이용하면 cpu의 다중 코어에 의해 여러개의 크롬을 병렬적으로 띄워 작업이 가능하게 됩니다.
* 예전의 cpu는 single core 혹은 dual core 였지만 현대의 cpu는 하드웨어에 대한 비약적인 발전으로 인해 multi core의 형태를 띄고 있습니다.
* 쓰레드는 한 프로세스 안에서 데이터를 공유하기 때문에 뉴스 제목, 뉴스기사, 뉴스 발행 날짜가 저장 될 수 있는 공유 리스트를 생성해서 각 쓰레드들이 생성하는 데이터를 실시간으로 합치는 방법을 채택 했었으나 이러한 경우에 여러개의 쓰레드가 뉴스제목, 뉴스기사, 뉴스 발행 날짜를 병렬적으로 append 할 경우 뉴스 제목,뉴스 기사, 뉴스 발행 날짜의 순서들이 꼬일 우려가 있기에 이를 각각의 쓰레드에서 지역 데이터 영역에서 리스트를 생성 후 쓰레드의 작업이 모두 끝나면 프로세스 공유 리스트에 한번에 추가하는 방법을 채택 하였습니다. 

In [ ]:
# 병렬로 수행될 크롤링 함수
def crawl(keyword, start_date, end_date):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-extensions")
    options.add_argument('--disable-dev-shm-usage')#특히 메모리 제한이 엄격한 환경이나, /dev/shm의 크기를 쉽게 조절할 수 없는 경우에 유용합니다. 이렇게 함으로써, 크롬의 안정성을 높이고 메모리 관련 문제를 방지할 수 있습니다.
    
    driver = webdriver.Chrome(options=options)

    global global_new_doc, global_publish_date, global_title_list, global_error_url_list
    
    url = f"https://search.naver.com/search.naver?where=news&query={keyword}&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds={start_date}&de={end_date}&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20220501to20220731&is_sug_officeid=0&office_category=0&service_area=2"
    driver.get(url)
    endScrollDown(driver)
    titles = driver.find_elements(By.CLASS_NAME, 'news_tit')
    
    url_list = []  # URL을 저장할 리스트
    local_new_doc=[]
    local_title_list= []
    local_publish_date= []
    local_error_url_list= []

    
    for i, title_element in enumerate(titles):
        try:
            url_list.append(title_element.get_attribute('href'))  # titles 리스트에서 URL을 추출해 url_list에 추가
        except:
            print("오류 발생")
            continue  # 오류가 발생해도 다음 항목으로 계속 진행
    
    # 뉴스 글 크롤링
    for news in notebook.tqdm(url_list):
        try:
            
            article= Article(news)
            article.download()
            article.parse()
            local_title_list.append(article.title)
            local_new_doc.append(article.text)
            local_publish_date.append(article.publish_date)
            
        except:
            local_error_url_list.append(news)
            continue

    global_title_list.extend(local_title_list)
    global_new_doc.extend(local_new_doc)
    global_publish_date.extend(local_publish_date)
    global_error_url_list.extend(local_error_url_list)

    driver.quit() # 크롬 닫기

# 스크롤 다운 함수에 driver 인자 추가
def endScrollDown(driver):
    end = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(0.5)
        start = driver.execute_script("return document.body.scrollHeight")
        if start == end:
            break
        end = start

# 메인 코드
if __name__ == "__main__": 
    global_new_doc = [] # 본문을 저장할 공유 리스트 초기화
    global_publish_date= [] # 신문 작성 날짜 저장할 공유 리스트 초기화
    global_title_list = []  # 제목을 저장할 공유 리스트
    global_error_url_list=[]
    start_time= datetime.now()

    
    keyword = '국민연금'
    start_date = ['2022.05.01', '2022.06.01', '2022.07.01', '2022.08.01', '2022.09.01', '2022.10.01', '2022.11.01', '2022.12.01', 
              '2023.01.01', '2023.02.01', '2023.03.01', '2023.04.01', '2023.05.01', '2023.06.01', '2023.07.01', '2023.08.01', 
              '2023.09.01', '2023.10.01', '2023.11.01', '2023.12.01', '2024.01.01', '2024.02.01', '2024.03.01', '2024.04.01']

    end_date =   ['2022.05.31', '2022.06.30', '2022.07.31', '2022.08.31', '2022.09.30', '2022.10.31', '2022.11.30', '2022.12.31', 
              '2023.01.31', '2023.02.28', '2023.03.31', '2023.04.30', '2023.05.31', '2023.06.30', '2023.07.31', '2023.08.31', 
              '2023.09.30', '2023.10.31', '2023.11.30', '2023.12.31', '2024.01.31', '2024.02.29', '2024.03.31', '2024.04.30']


    threads = []
    for i in range(len(start_date)):
        thread = Thread(target=crawl, args=(keyword, start_date[i], end_date[i]))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()  # 모든 스레드가 종료될 때까지 대기

    end_time= datetime.now()
    print("걸린시간: ", end_time - start_time)


In [ ]:
# 병렬로 수행될 크롤링 함수
def crawl(keyword, start_date, end_date):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-extensions")
    options.add_argument('--disable-dev-shm-usage')#특히 메모리 제한이 엄격한 환경이나, /dev/shm의 크기를 쉽게 조절할 수 없는 경우에 유용합니다. 이렇게 함으로써, 크롬의 안정성을 높이고 메모리 관련 문제를 방지할 수 있습니다.
    
    driver = webdriver.Chrome(options=options)

    global global_new_doc, global_publish_date, global_title_list, global_error_url_list
    
    url = f"https://search.naver.com/search.naver?where=news&query={keyword}&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds={start_date}&de={end_date}&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20220501to20220731&is_sug_officeid=0&office_category=0&service_area=2"
    driver.get(url)
    endScrollDown(driver)
    titles = driver.find_elements(By.CLASS_NAME, 'news_tit')
    
    url_list = []  # URL을 저장할 리스트
    local_new_doc=[]
    local_title_list= []
    local_publish_date= []
    local_error_url_list= []

    
    for i, title_element in enumerate(titles):
        try:
            url_list.append(title_element.get_attribute('href'))  # titles 리스트에서 URL을 추출해 url_list에 추가
        except:
            print("오류 발생")
            continue  # 오류가 발생해도 다음 항목으로 계속 진행
    
    # 뉴스 글 크롤링
    for news in notebook.tqdm(url_list):
        try:
            
            article= Article(news)
            article.download()
            article.parse()
            local_title_list.append(article.title)
            local_new_doc.append(article.text)
            local_publish_date.append(article.publish_date)
            
        except:
            local_error_url_list.append(news)
            continue

    global_title_list.extend(local_title_list)
    global_new_doc.extend(local_new_doc)
    global_publish_date.extend(local_publish_date)
    global_error_url_list.extend(local_error_url_list)

    driver.quit() # 크롬 닫기

# 스크롤 다운 함수에 driver 인자 추가
def endScrollDown(driver):
    end = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(0.5)
        start = driver.execute_script("return document.body.scrollHeight")
        if start == end:
            break
        end = start

# 메인 코드
if __name__ == "__main__": 
    global_new_doc = [] # 본문을 저장할 공유 리스트 초기화
    global_publish_date= [] # 신문 작성 날짜 저장할 공유 리스트 초기화
    global_title_list = []  # 제목을 저장할 공유 리스트
    global_error_url_list=[]
    start_time= datetime.now()

    
    keyword = '국민연금'
    start_date = ['2022.05.01', '2022.06.01', '2022.07.01', '2022.08.01', '2022.09.01', '2022.10.01', '2022.11.01', '2022.12.01', 
              '2023.01.01', '2023.02.01', '2023.03.01', '2023.04.01', '2023.05.01', '2023.06.01', '2023.07.01', '2023.08.01', 
              '2023.09.01', '2023.10.01', '2023.11.01', '2023.12.01', '2024.01.01', '2024.02.01', '2024.03.01', '2024.04.01']

    end_date =   ['2022.05.31', '2022.06.30', '2022.07.31', '2022.08.31', '2022.09.30', '2022.10.31', '2022.11.30', '2022.12.31', 
              '2023.01.31', '2023.02.28', '2023.03.31', '2023.04.30', '2023.05.31', '2023.06.30', '2023.07.31', '2023.08.31', 
              '2023.09.30', '2023.10.31', '2023.11.30', '2023.12.31', '2024.01.31', '2024.02.29', '2024.03.31', '2024.04.30']


    threads = []
    for i in range(len(start_date)):
        thread = Thread(target=crawl, args=(keyword, start_date[i], end_date[i]))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()  # 모든 스레드가 종료될 때까지 대기

    end_time= datetime.now()
    print("걸린시간: ", end_time - start_time)


  0%|          | 0/1568 [00:00<?, ?it/s]

  0%|          | 0/1721 [00:00<?, ?it/s]

  0%|          | 0/1819 [00:00<?, ?it/s]

  0%|          | 0/1813 [00:00<?, ?it/s]

  0%|          | 0/1872 [00:00<?, ?it/s]

...

  0%|          | 0/3029 [00:00<?, ?it/s]

  0%|          | 0/3080 [00:00<?, ?it/s]

  0%|          | 0/3346 [00:00<?, ?it/s]

  0%|          | 0/3682 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

### **12개의 쓰레드를 사용하여 1시간 18분(78분)이 소요됐다. 만약 멀티쓰레딩을 사용하지 않았다면 약 15시간(36분)이 소요 되었을 것이다.**

In [4]:
print("걸린시간: ", end_time - start_time)

걸린시간:  2:16:41.973516


#### **58174 개의 url 중 error가 발생하여 수집 진행되지 않은 url 갯수 확인**

In [8]:
print("error 발생으로 인한 수집 되지 않은 url:", len(global_error_url_list))

error 발생으로 인한 수집 되지 않은 url: 434


In [9]:
print(len(global_title_list))

print(len(global_new_doc))

print(len(global_publish_date))

57740
57740
57740


#### **뉴스 제목에 NULL 값은 없다**

In [11]:
print("뉴스 본문에 존재하는 NA 갯 수:",pd.isna(global_title_list).sum())

뉴스 본문에 존재하는 NA 갯 수: 0


In [12]:
print("뉴스 본문에 존재하는 NA 갯 수:",pd.isna(global_new_doc).sum())

뉴스 본문에 존재하는 NA 갯 수: 0


#### **뉴스 발행 날짜 NA 값은 갯수 확인**

In [20]:
print("뉴스 발행 날짜에 존재하는 NA 갯 수:",pd.isna(global_publish_date).sum())

뉴스 발행 날짜에 존재하는 NA 갯 수: 10743


### **에러가 발생한 url에 대해서 한번더 뉴스 추출**

In [21]:
global_error_url_list_again= [] # 한 번더 에러가 발생하는 url을 저장할 리스트 
count = 0 #얼마나 진행됐는지 확인하기 위한 변수 초기화
for news in global_error_url_list:
        try:
            article= Article(news)
            article.download()
            article.parse()
            global_publish_date.append(article.publish_date) #뉴스 발행 날짜 추가
            global_new_doc.append(article.text)              #뉴스 본문 내용 추가
            global_title_list.append(article.title)          #뉴스 제목 추가
        except:
            global_error_url_list_again.append(news)         #한번더 발생한 error url 추가
            continue
        count= count +1
        if count % 100 ==0:
            print(count)

100
200


In [22]:
print(len(global_title_list))

print(len(global_new_doc)) 

print(len(global_publish_date))

58016
58016
58016


#### **error가 발생하여 수집 진행되지 않은 url 갯수 확인**

In [23]:
print("error 발생으로 인한 수집 되지 않은 url:", len(global_error_url_list_again))

error 발생으로 인한 수집 되지 않은 url: 158


### **에러가 발생한 url에 대해서 또 한번 더 뉴스 추출**

In [24]:
global_error_url_list_again_2= []
count = 0
for news in global_error_url_list_again:
        try:
            article= Article(news)
            article.download()
            article.parse()
            global_publish_date.append(article.publish_date)
            global_new_doc.append(article.text)
            global_title_list.append(article.title)
        except:
            global_error_url_list_again_2.append(news)
            continue
        count= count +1
        if count % 10 ==0:
            print(count)

10
20
30
40
50
60
70


#### **error가 발생하여 수집 진행되지 않은 url 갯수 확인**

In [25]:
print("error 발생으로 인한 수집 되지 않은 url:", len(global_error_url_list_again_2))

error 발생으로 인한 수집 되지 않은 url: 85


In [26]:
print(len(global_title_list))

print(len(global_new_doc))

print(len(global_publish_date))

58089
58089
58089


In [27]:
pd.isna(global_new_doc).sum() # 뉴스 본문 데이터 null 값 갯 수

0

In [28]:
pd.isna(global_publish_date).sum() #발행 날짜 데이터 null 값 갯 수

10755

In [32]:
# 크롤링한 데이터를 데이터프레임으로 변환
pension = pd.DataFrame() 
pension['title'] = global_title_list
pension['doc'] = global_new_doc
pension['published_date']= global_publish_date
pension['ch'] = 'naver'

In [33]:
pension

,title,doc,published_date,ch
0,실직 중 지역가입자…국민연금 보험료 50%·최대 12개월 지원,[이데일리 양희동 기자] 보건복지부는 다음달 1일부터 국민연금 지역가입자 중 사업중...,2022-06-30 10:17:29+09:00,naver
1,"국민연금, 4월 말 수익률 -3.79%…전월보다 악화","신혜선 ""인생은 길고 역할은 많아요""",2022-06-29 15:36:43+09:00,naver
2,[포착] “상식 있는 정부라면 건강보험·국민연금 책임부터 다해야”,30일 오전 서울 용산 대통령 집무실 앞에서 열린 ‘국민연금-건강보험 국가 책임 강...,2022-06-30 12:08:35+09:00,naver
3,"‘더 내고 덜 받는’ 국민연금 개혁, 찬성 48%·반대 45%[NBS]",,2022-06-30 00:00:00,naver
4,"연금 200만원 받는 70대 피부양자, 9월부턴 건보료 낸다…얼마",정부가 올해 9월부터 소득이 많은데도 보험료를 내지 않는 건강보험 피부양자 27만3...,2022-06-29 17:23:07+09:00,naver
...,...,...,...,...
58084,"신한금융, 주총서 진옥동 회장 선임안 의결",,2023-03-23 00:00:00,naver
58085,"하나금융 주총, 이승열 행장 비상임이사 선임…분기배당 위한 정관변경","하나금융 주총, 이승열 행장 비상임이사 선임…분기배당 위한 정관변경\n\n입력: 2...",2023-03-24 13:42:45+09:00,naver
58086,1월 국세 수입 6.8조원 감소…나라살림은 7.3조원 흑자,,2023-03-16 00:00:00,naver
58087,[비즈토크<상>] 닻 올리기 전 몰아치는 폭풍우…KT 윤경림호 무사히 출범할까,[비즈토크<상>] 닻 올리기 전 몰아치는 폭풍우…KT 윤경림호 무사히 출범할까\n\...,2023-03-12 12:00:02+09:00,naver


In [ ]:
# 데이터프레임을 pickle파일로 저장
f = open("../data/raw_data/pension_news.pkl", "wb")
pickle.dump(pension,f)
f.close()

In [43]:
sum(pension['doc']=="")

5958

In [44]:
sum(pension['title']=="")

212